# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Part 1
### Instructions:
<ol>
    <li>Start by creating a new Notebook for this assignment.</li>
    <li>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe</li>
    <li>To create the above dataframe:<ul>
        <li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
        <li>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</li>
        <li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</li>
        <li>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</li>
        <li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
        <li>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</li>
        </ul></li>
    <li>Submit a link to your Notebook on your Github repository.</li>
</ol>

#### Start by Importing Tools

In [1]:
# Import tools

import numpy as np # library to handle data in a vectorized manner

import pandas as pd

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


#### Retrieve the Data

In [2]:
# Specify URL and identify page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Use Pandas to read HTML to return dataframe of dataframes
dfs = pd.read_html(url)

# First dataframe is the one I want.
df = dfs[0]

#### Next, QC the Data

In [3]:
# Check for "Not assigned" Boroughs
print(df.loc[df['Borough'] == 'Not assigned'])

# Remove "Not assigned" Boroughs
df = df[df.Borough != 'Not assigned']

    Postal Code       Borough Neighbourhood
0           M1A  Not assigned  Not assigned
1           M2A  Not assigned  Not assigned
7           M8A  Not assigned  Not assigned
10          M2B  Not assigned  Not assigned
15          M7B  Not assigned  Not assigned
16          M8B  Not assigned  Not assigned
19          M2C  Not assigned  Not assigned
24          M7C  Not assigned  Not assigned
25          M8C  Not assigned  Not assigned
28          M2E  Not assigned  Not assigned
29          M3E  Not assigned  Not assigned
33          M7E  Not assigned  Not assigned
34          M8E  Not assigned  Not assigned
35          M9E  Not assigned  Not assigned
37          M2G  Not assigned  Not assigned
38          M3G  Not assigned  Not assigned
42          M7G  Not assigned  Not assigned
43          M8G  Not assigned  Not assigned
44          M9G  Not assigned  Not assigned
51          M7H  Not assigned  Not assigned
52          M8H  Not assigned  Not assigned
53          M9H  Not assigned  N

In [4]:
# Check for additional "Not assigned" Neighborhoods; switch to US English spelling
print('"Not assigned" Neighborhoods:', df.loc[df['Neighbourhood'] == 'Not assigned'])

df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

"Not assigned" Neighborhoods: Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Note: No additional "Not assigned" Neighborhoods that weren't already addressed, so no further action required

In [5]:
# Check for multiple neighborhoods using the same postal code by looking for duplicate postal codes
print('Unique values:', df['Postal Code'].nunique(), 'Total Rows:', len(df.index))

Unique values: 103 Total Rows: 103


Note: No duplicate postal codes that weren't already addressed, so no further action required.

In [6]:
# Reset index to accomodate for dropped rows
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Show Dataframe Shape

In [7]:
df.shape

(103, 3)

## Part 2
### Instructions:
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

...we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create a  dataframe.

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the dataframe, submit a link to the new Notebook on your Github repository.

#### I chose to use the csv file option instead of the unreliable Geocoder method.

In [8]:
# Specify URL and identify page
url2 = 'http://cocl.us/Geospatial_data'

# Use Pandas to read csv into a dataframe
df2 = pd.read_csv(url2)

df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Inner join to merge two dataframes
df3 = pd.merge(left=df, right=df2)

df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
df3.shape

(103, 5)

## Part 3
### Instructions:
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:
<ol>
    <li>to add enough Markdown cells to explain what you decided to do and to report any observations you make.</li>
    <li>to generate maps to visualize your neighborhoods and how they cluster together.</li>
</ol>    
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository.

### Explore Neighborhoods in Toronto
#### Start by filtering for only boroughs containing "Toronto"

In [11]:
df4 = df3[df3["Borough"].str.contains('Toronto')]
df4.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Use geopy library to get the latitude and longitude values of Toronto

In [12]:
address = 'Toronto, Ontario, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Create map of Toronto with  neighborhoods superimposed on top.

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#CC8899',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Narrow down to one Borough and map it

In [14]:
borough = 'Central Toronto'
df5 = df4[df4['Borough'] == borough].reset_index(drop=True)
df5.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


In [15]:
address = borough + ', Ontario, CA'

geolocator = Nominatim(user_agent="borough_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of borough using latitude and longitude values
map_borough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#CC8899',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough)  
    
map_borough

#### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'XBM5I2IHZZTLDUAZW5FOO1OFMSA0WLFGGJMGRSHWVHCHYTVY' # your Foursquare ID
CLIENT_SECRET = 'H2XRBU1HAD2EIPKIXPOKOXPXT3OITIH40TOJNOSKZFUX435D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Define function to retrieve the top 100 venues in a 500m radius

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # creates the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # makes the get request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # returns only relevant information for each close venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Call the method

In [18]:
venues = getNearbyVenues(names=df5['Neighborhood'],
                                   latitudes=df5['Latitude'],
                                   longitudes=df5['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


#### Get some information about the resulting dataframe

In [19]:
# Shape and Size of dataframe
print(venues.shape)
venues.head()

(112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


In [20]:
# Venues returned for each neighborhood
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,34,34,34,34,34,34
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",5,5,5,5,5,5
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",3,3,3,3,3,3
"North Toronto West, Lawrence Park",19,19,19,19,19,19
Roselawn,1,1,1,1,1,1
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",21,21,21,21,21,21


In [21]:
# Number of unique categories from all returned venues
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 61 uniques categories.


### Analyze Each Neighborhood

In [22]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood  by taking the mean of the frequency of occurrence of each category

In [23]:
grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0000,0.029412,0.000000,0.0000,0.0000,0.000000,0.029412,0.000000,0.00,...,0.0000,0.0000,0.058824,0.00,0.029412,0.029412,0.000000,0.000000,0.0000,0.000000
1,Davisville North,0.0000,0.000000,0.000000,0.0000,0.0000,0.111111,0.000000,0.000000,0.00,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.20,...,0.0000,0.0000,0.200000,0.00,0.000000,0.000000,0.200000,0.000000,0.0000,0.000000
3,Lawrence Park,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.25,...,0.0000,0.0000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,"Moore Park, Summerhill East",0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.0000,0.000000
5,"North Toronto West, Lawrence Park",0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.052632
6,Roselawn,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.0625,0.000000,0.000000,0.0625,0.0625,0.000000,0.000000,0.000000,0.00,...,0.0625,0.0625,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000
8,"The Annex, North Midtown, Yorkville",0.0000,0.000000,0.047619,0.0000,0.0000,0.000000,0.000000,0.047619,0.00,...,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.0000,0.000000


#### Print each neighborhood and it's top 5 most common venues

In [24]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.12
1      Sandwich Place  0.09
2         Pizza Place  0.09
3         Coffee Shop  0.06
4  Italian Restaurant  0.06


----Davisville North----
                  venue  freq
0                 Hotel  0.11
1      Department Store  0.11
2  Gym / Fitness Center  0.11
3                   Gym  0.11
4                  Park  0.11


----Forest Hill North & West, Forest Hill Road Park----
              venue  freq
0          Bus Line   0.2
1     Jewelry Store   0.2
2             Trail   0.2
3              Park   0.2
4  Sushi Restaurant   0.2


----Lawrence Park----
                venue  freq
0  Dim Sum Restaurant  0.25
1         Swim School  0.25
2            Bus Line  0.25
3                Park  0.25
4  Salon / Barbershop  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Restaurant  0.33
1                Trail  0.33
2                 Park  0.33
3  American Restaurant  0.00
4   Italian Resta

#### Convert to pandas dataframe

In [25]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
# Create dataframe and display top 10 venues per neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Gym,Italian Restaurant,Greek Restaurant,Gourmet Shop
1,Davisville North,Hotel,Sandwich Place,Food & Drink Shop,Park,Pizza Place,Gym / Fitness Center,Department Store,Gym,Breakfast Spot,Burger Joint
2,"Forest Hill North & West, Forest Hill Road Park",Jewelry Store,Park,Trail,Sushi Restaurant,Bus Line,Flower Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
3,Lawrence Park,Dim Sum Restaurant,Swim School,Bus Line,Park,Yoga Studio,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint
4,"Moore Park, Summerhill East",Restaurant,Trail,Park,Yoga Studio,Flower Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop


### Cluster Neighborhoods
#### Run k-means to cluster neighborhood into 5 clusters

In [27]:
# Set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 4, 0, 1, 0, 0], dtype=int32)

#### Create new dataframe including the cluster as well as top 10 venues for each neighborhood.

In [28]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df5

# merge grouped with data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Dim Sum Restaurant,Swim School,Bus Line,Park,Yoga Studio,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop,Flower Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Sandwich Place,Food & Drink Shop,Park,Pizza Place,Gym / Fitness Center,Department Store,Gym,Breakfast Spot,Burger Joint
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Jewelry Store,Park,Trail,Sushi Restaurant,Bus Line,Flower Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Park,Pet Store


#### Visualize clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
#### Cluster 1

In [30]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Hotel,Sandwich Place,Food & Drink Shop,Park,Pizza Place,Gym / Fitness Center,Department Store,Gym,Breakfast Spot,Burger Joint
4,Central Toronto,0,Clothing Store,Coffee Shop,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Park,Pet Store
5,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Indian Restaurant,Park,Liquor Store,Pharmacy,Pub,Donut Shop,Pizza Place
6,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Gym,Italian Restaurant,Greek Restaurant,Gourmet Shop
8,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Sports Bar,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store,Restaurant,Pizza Place


#### Cluster 2

In [31]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Garden,Yoga Studio,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gas Station,Fried Chicken Joint,Food & Drink Shop,Flower Shop


#### Cluster 3

In [32]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,2,Jewelry Store,Park,Trail,Sushi Restaurant,Bus Line,Flower Shop,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio


#### Cluster 4

In [33]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,3,Dim Sum Restaurant,Swim School,Bus Line,Park,Yoga Studio,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint


#### Cluster 5

In [34]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,4,Restaurant,Trail,Park,Yoga Studio,Flower Shop,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop
